In [8]:
import os

from dotenv import load_dotenv

load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

In [9]:
from langchain_text_splitters import Language, RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain_community.document_loaders.generic import GenericLoader
from langchain_community.document_loaders.parsers import LanguageParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

from git import Repo

In [10]:
repo_path = './test_repo'

In [11]:
repo = Repo.clone_from('https://github.com/openai-php/client', to_path=repo_path)

In [12]:
loader = GenericLoader.from_filesystem(
  repo_path + '/src/Resources/',
  glob='**/*',
  suffixes=['.php'],
  exclude=['!non-php'],
  parser=LanguageParser(Language.PHP, parser_threshold=500)
)

documents = loader.load()
len(documents)


22

In [13]:
python_splitter = RecursiveCharacterTextSplitter.from_language(
  Language.PHP, 
  chunk_size=2000,
  chunk_overlap=200
)

texts = python_splitter.split_documents(documents)
len(texts)

58

In [14]:
db = Chroma.from_documents(texts, OpenAIEmbeddings(disallowed_special=()))

retriever = db.as_retriever(
  search_types='mmr',
  search_kwargs={'k': 8}
)

In [15]:
llm = ChatOpenAI(model='gpt-3.5-turbo', max_tokens=200)

In [16]:
prompt = ChatPromptTemplate.from_messages(
  [
    (
      'system',
      '''
      Você é um revisor de código experiente. 
      Forneça informações detalhadas sobre a revisão do código e sugestões de melhorias 
      baseado no contexto fornecido abaixo: \n\n {context}
      '''
    ),
    (
      'user',
      '{input}'
    )
  ]
)

document_chain = create_stuff_documents_chain(
  llm, prompt
)

retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [17]:
response = retrieval_chain.invoke(
  {'input': 'Você pode revisar e sugerir melhorias para o código de HttpTransporter?'}
)

In [18]:
print(response['answer'])

### Revisão do código de HttpTransporter:

1. **Método `requestContent`**:
   - O método `requestContent` parece não estar completo. Não está claro o que ele deve fazer, pois não há documentação ou implementação do retorno.
   - Sugiro adicionar documentação detalhada sobre o propósito e o comportamento esperado do método.
   - Verifique se o método está implementado corretamente de acordo com a lógica desejada.

2. **Método `upload`**:
   - No método `upload`, a chamada para criar um novo `CreateResponse` está bem estruturada e clara.
   - No entanto, sugiro adicionar mais comentários explicativos sobre o que está sendo feito em cada etapa do método para facilitar a compreensão.

3. **Método `delete`**:
   - O método `delete` está bem
